# Knapsack Problem

Statchel can only hold C lbs. of treasure. There are n items in the treasure chest. Each item i: value vi and a weight wi.

**Which items should be put in the satchel to maximize value?**

In the Project:

• 500 Knapsack Instances

• C in U[100,150]

• Only find out C once you choose which items to put in the knapsack

• If you go over C, then you get nothing for that instance

## 1. Use Dynamic Programming to Maximize the Value of a Given Bag Size:

In [1]:
import numpy as np

def solution(i,j,wt,val,track):
    if track[int(i)][int(j)] != []:
        return track[int(i)][int(j)]
    if i == 0 or j == 0:
        ans = 0
    elif wt[i-1] > j:
        ans = solution(i-1, j,wt,val,track)
    else:
        option_1 = solution(i-1, j,wt,val,track)
        option_2 = solution(i-1, j-wt[i-1],wt,val,track)+ val[i-1]
        ans =  max(option_1,option_2)    
    track[int(i)][int(j)] = ans 
    return ans
    
def knapsack(bag_size,wt,val):
    i = 50 # 50 items in each instance
    j = bag_size
    track = [[]]*(i+1)
    for i in range(i+1):
        track[i] = [[]]*(j+1)
    for a in range(i+1):
        track[a][0] = 0
    for b in range(j+1):
        track[0][b] = 0
    
    sum = solution(i,j,wt,val,track)
    return sum   

In [2]:
# Read weight and value of each item without using Pandas

def read_weight(k):
    csv_name = "./Knapsack_Instances.csv"
    csv = open(csv_name,"r")
    lines = csv.readlines()
    
    wt = lines[3*k].split(",")
    for string in range(len(wt)):    
        wt[string] = float(wt[string])
    wt = wt[:50]
    return wt
    
def read_value(k):
    csv_name = "./Knapsack_Instances.csv"
    csv = open(csv_name,"r")
    lines = csv.readlines()
    
    val = lines[3*k+1].split(",")
    for string in range(len(val)):
        val[string] = float(val[string])
    val = val[:50]
    return val

## 2. Use Monte Carlo to Find a Good Mix of Knapsack Sizes: 

In [8]:
# Create all possible target bag size

bag_interval = []
for i in range(51):
    bag_interval.append(i+100)
    
print(bag_interval)

[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150]


In [ ]:
store_best_size_for_all_instance = []

for k in range(500): # 500 instances
    
    wt = read_weight(k)
    val = read_value(k)
    
    store_average_difference_for_all_target_size = []
    
    for r in range(len(bag_interval)): # C in U[100,150]
  
        target_bag_size = bag_interval[r]  
        
        # find the optimal value given a fixed bag size
        target_bag_size_value_original = knapsack(target_bag_size,wt,val)
        
        # set number of simulations 
        num_of_simulations = 50000
        
        # create random bag sizes for the simulation
        random_size = np.random.randint(100,151,num_of_simulations)

        simulation_result_of_actual_bag_size = {}

        total_difference_for_simulation_of_each_target = []

        # SIMULATION: 
        for i in range(num_of_simulations):
            
            actual_bag_size = random_size[i]

            target_bag_size_value = target_bag_size_value_original

            if target_bag_size > actual_bag_size: # If you go over C, then you get nothing for that instance
                target_bag_size_value = 0

            if actual_bag_size in simulation_result_of_actual_bag_size: # Use previously calculated value
                actual_bag_size_value = simulation_result_of_actual_bag_size[actual_bag_size]
            else:
                actual_bag_size_value = knapsack(actual_bag_size,wt,val)
                simulation_result_of_actual_bag_size[actual_bag_size] = actual_bag_size_value # Record the size & value

            # Compare the difference of value between target bag size & actual bag size 
            difference_bw_actual_and_target = actual_bag_size_value - target_bag_size_value
            total_difference_for_simulation_of_each_target.append(difference_bw_actual_and_target)
            
        # End of Simulation
        
        # Calculate the mean difference & store the mean difference for that target size
        average_difference_bw_actual_and_target_for_each_target = np.mean(total_difference_for_simulation_of_each_target)
        store_average_difference_for_all_target_size.append(average_difference_bw_actual_and_target_for_each_target)
    
    # Get the best value & size for that instance (a total of 500 instances)
    best_value_for_this_instance = np.min(store_average_difference_for_all_target_size)
    best_size_for_this_instance = 100 + store_average_difference_for_all_target_size.index(np.min(store_average_difference_for_all_target_size))
    print('the best value for instance',(k+1),'is',best_value_for_this_instance)
    print('the best size for instance',(k+1),'is',best_size_for_this_instance)
    
    # Store the best size by appending to the list 
    store_best_size_for_all_instance.append(best_size_for_this_instance)

# Get the best sizes for all instances
print(store_best_size_for_all_instance)